# Imports

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import numpy as np
import numpy.matlib
import pandas as pd
import time
import os
from os import listdir
from os.path import join, isfile
import datetime

!pip install jupyter-dash
from jupyter_dash import JupyterDash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import requests
import io
from plotly.subplots import make_subplots

!pip install dash-bootstrap-components<1
import dash
import dash_bootstrap_components as dbc
!pip install dash-bootstrap-templates
import dash_bootstrap_templates
from dash_bootstrap_templates import load_figure_template
from dash_bootstrap_templates import ThemeChangerAIO, template_from_url

# Data Processing

In [ ]:
path = 'https://github.com/BobbyWilt/PD-Watch-Dashboard/raw/main/Data/data/110001/'

cols = ['left_acc_x', 'left_acc_y', 'left_acc_z',
                'left_gyr_x', 'left_gyr_y', 'left_gyr_z',
                'right_acc_x', 'right_acc_y', 'right_acc_z',
                'right_gyr_x', 'right_gyr_y', 'right_gyr_z',
                'chest_acc_x', 'chest_acc_y', 'chest_acc_z',
                'chest_gyr_x', 'chest_gyr_y', 'chest_gyr_z'
               ]
# Loop through files and load csv's into dataframe.  Reset columns and index.  Add start and end times to a list
rec_times = []
for i in range(1, 8):
    url = path + f'data_segment_{i}.csv'
    locals()[f'df{i}'] = pd.read_csv(url)
    locals()[f'df{i}'].set_index('time', inplace=True)
    locals()[f'df{i}'].columns = cols
    start = locals()[f'df{i}'].index[0].split(' ')[1][:-7]
    end = locals()[f'df{i}'].index[-1].split(' ')[1][:-7]
    rec_times.append([start,end])
    

# Figures

In [ ]:
# Set default figure template
load_figure_template('DARKLY')
# Make loop to make all time-series plots
for i in range(1,8):
    # Mulit-line plot - separated by sensor
    locals()[f'fig{i}'] = make_subplots(rows=6, cols=1, shared_xaxes=True,
                                        vertical_spacing=0.06,
                                        subplot_titles=[
                                            'Accelerometry Left', 'Gyro Left',
                                            'Accelerometry Right', 'Gyro Right',
                                            'Accelerometry Chest', 'Gyro Chest'])
    data = locals()[f'df{i}']
    sensor_axis_types = ['acc_x', 'acc_y', 'acc_z',
                         'gyr_x', 'gyr_y', 'gyr_z']

    count, count_l = 1, 0
    # Loop thorugh each sensor
    for sens in ['left', 'right', 'chest']:
    # Loop through each sensor data type
        for d_type in ['acc', 'gyr']:
            # loop through each sensor axis
            for axis, color in zip(['x', 'y', 'z'],
                                   ['MediumSeaGreen', 'SandyBrown', 'DodgerBlue']):
                count_l+=1
                if count_l > 3:
                    legend_vis=False
                else:
                    legend_vis=True

                locals()[f'fig{i}'].add_trace(
                    go.Scatter(x=data.index, y=data[f'{sens}_{d_type}_{axis}'],
                               marker_color=color, name=f'{axis}-axis', showlegend=legend_vis),
                    row=count, col=1)
            count += 1

#     locals()[f'fig{i}'].update_layout(height=1500, width=1500, title_text=f"Data Segment {i}")
    locals()[f'fig{i}'].update_layout(height=1500, width=1300)

# Store figures in a list
ts_figures = [fig1, fig2, fig3, fig4, fig5, fig6, fig7]

In [ ]:
# Patient survey data
quest_url = 'https://raw.githubusercontent.com/BobbyWilt/PD-Watch-Dashboard/main/Data/data/110001/110001_features900.csv'
df_quest = pd.read_csv(quest_url)

# Patient questionairre questions
mood_cats = ['mood_well', 'mood_down', 'mood_fright', 'mood_tense',
             'phy_sleepy', 'phy_tired', 'mood_cheerf', 'mood_relax']
mood_nams = ['Well', 'Down', 'Frightened', 'Tense', 'Sleepy', 'Tired',
             'Cheerful', 'Relaxed']
symptom_cats = ['tremor', 'slowness', 'stiffness', 'muscle_tension', 'dyskinesia']
functionality_cats = ['act_problemless', 'mobility_well', 'sit_still',
                      'speech_well', 'walk_well']
functionality_nams = ['Current Activity', 'Mobility Comfort', 'Sit & Stand',
                      'Speech', 'Walk']

# Create dataframes
df_mood = df_quest[mood_cats]
df_sympt = df_quest[symptom_cats]
df_func = df_quest[functionality_cats]

# Dashboard

In [ ]:
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates@V1.0.2/dbc.min.css"
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])
app.layout = dbc.Container([
    html.H1("Parkinson's Patient Dashboard"),
    html.Hr(),

    dbc.Row([
        dbc.DropdownMenu(
                    label="Data Segment",
                    menu_variant="dark",
                    children=[
                        dbc.DropdownMenuItem("Segment 1", id='1'),
                        dbc.DropdownMenuItem("Segment 2", id='2'),
                        dbc.DropdownMenuItem("Segment 3", id='3'),
                        dbc.DropdownMenuItem("Segment 4", id='4'),
                        dbc.DropdownMenuItem("Segment 5", id='5'),
                        dbc.DropdownMenuItem("Segment 6", id='6'),
                        dbc.DropdownMenuItem("Segment 7", id='7'),
                    ],id='dropdown_item', className='mt-3'),
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Row([
                
                dbc.Col([
                    html.H3('Date'),
                    html.H5(id='date'),
                ]),
                dbc.Col([
                    html.H3('Watch Recording Time'),
                    html.H5(id='time_frame'),
                ]),
                dbc.Col([
                    html.H3('Survey Time'),
                    html.H5(id='time_surv'),
                ])
            ]),
            
        ], width=8),
        dbc.Col([
            html.H3('Medication Status'),
            dbc.Row([
                dbc.Col([
                html.H4(id='med_state'),
                ]),
#                 dbc.Col([
#                     html.H6('Med image'),
#                 ])
            ])  
        ], width=4)
    ]),
    
    dbc.Row([
        dbc.Col([
            html.H3('Time Series Graphs'),
            html.H5(id='data_segment'),
            dcc.Graph(id='time-series', className="w-100"),
        ], width=8),
        dbc.Col([
            dbc.Row([
                html.H3('Motor Symptoms'),
                dbc.Row([
                    html.H5('Symptoms'),
                    dcc.Graph(id='pd_symptoms', className='w-100'),
#                     html.Div('description of symptoms'),
                ]),
                dbc.Row([
                    html.H5('Functionality'),
                    dcc.Graph(id='functionality'),
#                     html.Div('description of functions'),
                ])
            ]),
            dbc.Row([
                html.H3('Cognitive Symptoms'),
                html.H5('Mood'),
                dcc.Graph(id='cog_symptoms'),
            ])
        ], width=4)
    ])
],
    className='dbc',
    fluid=True
)

# Data Segment Selector Callback
@app.callback(
    Output('data_segment', 'children'),
    Output('date', 'children'),
    Output('time_frame', 'children'),
    Output('time_surv', 'children'),
    Output('med_state', 'children'),
    Output('pd_symptoms', 'figure'),
    Output('functionality', 'figure'),
    Output('cog_symptoms', 'figure'),
    Output('time-series', 'figure'),
    [
        Input('1', 'n_clicks'),
        Input('2', 'n_clicks'),
        Input('3', 'n_clicks'),
        Input('4', 'n_clicks'),
        Input('5', 'n_clicks'),
        Input('6', 'n_clicks'),
        Input('7', 'n_clicks'),
    ]
)

def update_figs(*args):
    # Obtain data segment index
    ctx = dash.callback_context
    if not ctx.triggered:
        data_segment_index='1'
    else:
        data_segment_index = ctx.triggered[0]['prop_id'].split('.')[0]
    
    data_segment_index = int(data_segment_index)
    i = data_segment_index - 1
    
    out_label = f"Data Segment {data_segment_index}"
    # Get date and time frames
    date = df_quest['beep_time_start'][i].split(' ')[0]
    date = datetime.datetime(int(date[:4]), int(date[5:7]), int(date[8:10]))
    date = date.strftime("%b %d %Y")
    # recording time
    time_frame = f'{rec_times[i][0]} - {rec_times[i][1]}'
    # survey time
    start = df_quest['beep_time_start'][i].split(' ')[1][:-3]
    end = df_quest['beep_time_end'][i].split(' ')[1][:-3]
    time_surv = f"{start} - {end}"
    
    # Med status
    med_score = int(df_quest['sanpar_onoff'][i])
    if med_score==1:
        med_state='OFF'
    elif med_score==2:
        med_state='OFF to ON'
    elif med_score==3:
        med_state='ON'
    elif med_score==4:
        med_state='ON to OFF'
    else:
        med_state="None Entered"
        
    # PD_symptoms plot
    fig_sp_pd=go.Figure()
    fig_sp_pd.add_trace(
        go.Scatterpolar(r =df_sympt.iloc[i],
                        theta=symptom_cats, fill='toself')
    )
    fig_sp_pd.update_polars(radialaxis_range=(1,7))
    fig_sp_pd.update_layout(margin=dict(l=0, r=0, t=20, b=20))
    # functionality plot
    fig_f=go.Figure()
    fig_f.add_trace(
        go.Scatterpolar(r =df_func.iloc[i],
                        theta=functionality_nams, fill='toself')
    )
    fig_f.update_layout(margin=dict(l=0, r=0, t=20, b=20))
    fig_f.update_polars(radialaxis_range=(1,7))
    # Cognitive symptoms plot
    fig_sp_c = go.Figure()
    fig_sp_c.add_trace(
        go.Scatterpolar(name='Mood', r=df_mood.iloc[i],
                        theta=mood_nams,fill='toself')
    )
    fig_sp_c.update_polars(radialaxis_range=(1,7))
    fig_sp_c.update_layout(margin=dict(l=0, r=0, t=20, b=20))
    # time-series plot
    fig_ts = ts_figures[i]
    
    # return figures
    return out_label, date, time_frame, time_surv, med_state, fig_sp_pd, fig_f, fig_sp_c, fig_ts

app.run_server(debug=True)